# Pre


In [130]:
! pip install -U pandasql
import torch
import pandas as pd
import pandasql as ps
import numpy as np

In [131]:
from google.colab import drive 

In [132]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [133]:
%cd drive/MyDrive/tarak/Players-Block-20211116T025030Z-001/Players-Block/GNN/data/

[Errno 2] No such file or directory: 'drive/MyDrive/tarak/Players-Block-20211116T025030Z-001/Players-Block/GNN/data/'
/content/drive/.shortcut-targets-by-id/1MeacoHD16_nfxfWKec4Nch65B3Nxp4K-/tarak/Players-Block-20211116T025030Z-001/Players-Block/GNN/data


In [134]:
!ls

 Competes_in_edges.csv	  matches.csv
 Comp_season_nodes.csv	  matches_with_player_features.csv
 Field_player_nodes.csv   Plays_for_edges.csv
 GK_nodes.csv		  Plays_with_edges.csv
 gnn_data.docx		 'Team_nodes(1).csv'
 gnn_data.pdf		  Team_nodes.csv
 Hosts_edges.csv	  team_nodes_plus_features.csv
 Manager_nodes.csv	  team_players_game.csv
 Manages_edges.csv


In [135]:
PlaysWithEdgesDf = pd.read_csv('./Plays_with_edges.csv')
CompetesInEdgesDf = pd.read_csv('./Competes_in_edges.csv')
CompSeasonNodesDf = pd.read_csv ('Comp_season_nodes.csv')
FieldPlayerNodesDf = pd.read_csv('Field_player_nodes.csv')
GKNodesDf = pd.read_csv('GK_nodes.csv')
HostsEdgesDf = pd.read_csv('Hosts_edges.csv')
ManagerNodesDf = pd.read_csv('Manager_nodes.csv')
ManagesEdgesDf = pd.read_csv('Manages_edges.csv')
PlaysForEdgesDf = pd.read_csv('Plays_for_edges.csv')
TeamNodesDf = pd.read_csv('Team_nodes.csv')

# %cd ../../../../../../../../
# !ls
FeatureTeamNodesDf = pd.read_csv('team_nodes_plus_features.csv')

In [136]:
FeatureTeamNodesDf.columns

Index(['Unnamed: 0', 'teamName', 'postGame_side', 'competitionId', 'gameId',
       'teamId', 'preGame_steph', 'preGame_quality_overall',
       'preGame_quality_attack', 'preGame_quality_midfield',
       'preGame_quality_defence', 'preGame_proportion_pts', 'gameDate',
       'teamName.1', 'competitionName', 'nodeId', 'nodeName', 'year', 'month',
       'postGame_goals_scored', 'agg_postGame_goals_scored',
       'postGame_goals_conceded', 'agg_postGame_goals_conceded',
       'postGame_index', 'agg_postGame_index', 'postGame_total_shots',
       'agg_postGame_total_shots', 'postGame_total_xg',
       'agg_postGame_total_xg', 'postGame_total_big_chances',
       'agg_postGame_total_big_chances', 'postGame_total_key_passes',
       'agg_postGame_total_key_passes'],
      dtype='object')

In [137]:
FeatureTeamNodesDf = FeatureTeamNodesDf.fillna(0).reset_index()
FeatureTeamNodesDf.shape[0] // 2

4014

In [138]:
query = """ 
  select count(*)
  from FeatureTeamNodesDf T
  inner join HostsEdgesDf H
      on T.nodeName = H.dest_nodeName and T.gameId = H.gameId
"""
ps.sqldf(query)

,count(*)
0,4014


# Data for BC


In [ ]:
HostsEdgesDf.columns

Index(['gameId', 'homeTeamId', 'awayTeamId', 'gameDate', 'seasonId',
       'competitionId', 'competitionName', 'homeTeamName', 'awayTeamName',
       'preGame_venueId', 'preGame_city', 'preGame_country', 'preGame_lat',
       'preGame_long', 'preGame_field_length_m', 'preGame_field_width_m',
       'preGame_capacity', 'preGame_surface', 'preGame_is_behind_closed_doors',
       'preGame_homeTeamDominance', 'preGame_awayTeamDominance',
       'preGame_venueName', 'postGame_tgt_half_gd', 'postGame_tgt_gd',
       'postGame_tgt_homeTeamGoals', 'postGame_tgt_awayTeamGoals',
       'postGame_tgt_half_homeTeamGoals', 'postGame_tgt_half_awayTeamGoals',
       'postGame_tgt_outcome', 'postGame_tgt_half_outcome', 'preGame_odds1',
       'preGame_odds2', 'preGame_oddsX', 'src_nodeId', 'src_nodeName',
       'edge_type', 'dest_nodeId', 'dest_nodeName'],
      dtype='object')

In [ ]:
query = """ 
  select preGame_steph, preGame_quality_overall,
          preGame_quality_attack, preGame_quality_midfield,
          preGame_quality_defence, preGame_proportion_pts,
          agg_postGame_goals_scored,
          agg_postGame_goals_conceded,
          agg_postGame_index,
          agg_postGame_total_shots,
          agg_postGame_total_xg,
          agg_postGame_total_big_chances,
          agg_postGame_total_key_passes,
          preGame_odds1,
          preGame_odds2,
          preGame_oddsX,
          preGame_homeTeamDominance,
          preGame_awayTeamDominance
  from FeatureTeamNodesDf T inner join HostsEdgesDf H
    on T.nodeName = H.dest_nodeName
  order by T.gameDate, T.gameId
"""
away_features = ps.sqldf(query)
query = """ 
  select preGame_steph, preGame_quality_overall,
          preGame_quality_attack, preGame_quality_midfield,
          preGame_quality_defence, preGame_proportion_pts,
          agg_postGame_goals_scored,
          agg_postGame_goals_conceded,
          agg_postGame_index,
          agg_postGame_total_shots,
          agg_postGame_total_xg,
          agg_postGame_total_big_chances,
          agg_postGame_total_key_passes,
          preGame_odds1,
          preGame_odds2,
          preGame_oddsX,
          preGame_homeTeamDominance,
          preGame_awayTeamDominance
  from FeatureTeamNodesDf T inner join HostsEdgesDf H
    on T.nodeName = H.src_nodeName
  order by T.gameDate, T.gameId
"""
home_features = ps.sqldf(query)
ps.sqldf(query)

query = """
  select postGame_tgt_gd
  from HostsEdgesDf H inner join FeatureTeamNodesDf T
    on H.src_nodeName = T.nodeName
  order by T.gameDate, T.gameId
"""
results = ps.sqldf(query)

In [ ]:
results = torch.from_numpy(results.values).view(-1)
dummy = 6
results[results > 0] = dummy + 2
results[results == 0] = dummy + 1
results[results < 0] = dummy + 0
results -= dummy

home_features = torch.from_numpy(home_features.values)
away_features = torch.from_numpy(away_features.values)

# Blade Chest Model

In [ ]:
from torch.nn import Module, Linear, NLLLoss, Tanh, BatchNorm1d, Dropout, LogSoftmax, PairwiseDistance
from torch.optim import Adam
from torchsummary import summary

In [ ]:
class BladeChest(Module):
  def __init__(self, feature_size, hidden_size, dropout_rate=0.5):
    super(BladeChest, self).__init__()
    self.feature_size = feature_size
    self.hidden_size = hidden_size
    self.blade_transfrom = Linear(self.feature_size, self.hidden_size)
    self.chest_transform = Linear(self.feature_size, self.hidden_size)
    self.activation = Tanh()
    self.feature_bn = BatchNorm1d(self.feature_size)
    self.hidden_bn = BatchNorm1d(self.hidden_size)
    self.regularizer = Dropout(dropout_rate)
    self.result_transform = Linear(1, 3)
    self.distance_fn = PairwiseDistance()
    self.classifier = LogSoftmax(-1)

  def _matchup_score(self, away_blade, home_chest, home_blade, away_chest):
    return self.distance_fn(away_blade, home_chest).pow(2) - self.distance_fn(home_chest, away_blade).pow(2)

  def encode_team(self, x_team):
    h_team = self.feature_bn(x_team)

    team_blade = self.blade_transfrom(h_team)
    team_blade = self.hidden_bn(team_blade)
    team_blade = self.regularizer(team_blade)

    team_chest = self.chest_transform(h_team)
    team_chest = self.hidden_bn(team_chest)
    team_chest = self.regularizer(team_chest)

    return team_blade, team_chest

  def forward(self, x_home, x_away):
    home_blade, home_chest = self.encode_team(x_home)
    away_blade, away_chest = self.encode_team(x_away)
    h_home = self.feature_bn(x_home)
    h_away = self.feature_bn(x_away)

    matchup = self._matchup_score(away_blade, home_chest, home_blade, away_chest).view(-1, 1)
    result_h = self.result_transform(matchup)
    result_h = self.regularizer(result_h)

    return self.classifier(result_h)

In [ ]:
results.shape

torch.Size([4014])

In [ ]:
train_size = 3200
validation_size = 400
batch_size = 32
rand_indcs = torch.randperm(results.shape[0])
train_indcs = rand_indcs[:train_size]
val_indcs = rand_indcs[train_size: train_size + validation_size]
test_indcs = rand_indcs[train_size + validation_size:]
train_home_features = home_features[train_indcs, :].reshape(-1, batch_size, away_features.shape[-1])
train_away_features = away_features[train_indcs, :].reshape(-1, batch_size, away_features.shape[-1])
train_results = results[train_indcs].reshape(-1, batch_size)

val_home_features = home_features[val_indcs, :]
val_away_features = away_features[val_indcs, :]
val_results = results[val_indcs]
test_home_features = home_features[test_indcs, :]
test_away_features = away_features[test_indcs, :]
test_results = results[test_indcs]

In [ ]:
def train(model, home, away, results, optimizer, criterion):
  model.train()
  optimizer.zero_grad()
  output = model(home, away)
  loss = criterion(output, results)
  loss.backward()
  optimizer.step() 
  return loss.item()


@torch.no_grad()
def eval(model, home, away, results):
  model.eval()
  output = model(home, away).argmax(-1)
  correct = (output == results).sum().item()
  return (correct / results.shape[0])

In [ ]:
feature_size = home_features.shape[-1]
bc_model = BladeChest(feature_size, 128, 0.4).double()
######################################
scaled_class_weight = torch.tensor([1.5, 1.8, 1], dtype=torch.double)
#####################################
criterion = NLLLoss(scaled_class_weight)
print(f'Initial Random State Model: ')
print(f'Validation Acc: {eval(bc_model, val_home_features, val_away_features, val_results) * 100: .4f}%')
print(f'Test Acc: {eval(bc_model, test_home_features, test_away_features, test_results) * 100: .4f}%')

Initial Random State Model: 
Validation Acc:  24.5000%
Test Acc:  25.1208%


In [ ]:
optimizer = Adam(bc_model.parameters(), lr=1e-3, weight_decay=0)
try:
  for epoch in range(1500000):
    b_loss = 0
    for home, away, result in zip(train_home_features, train_away_features, train_results):
      b_loss += train(bc_model, home, away, result, optimizer, criterion)
    print(f'Epoch: {epoch + 1}')
    print(f'Batch Avg. Loss: {b_loss / batch_size: .4f}')
    print(f'Train Acc: {eval(bc_model, train_home_features.reshape(-1, feature_size), train_away_features.reshape(-1, feature_size), train_results.reshape(-1)) * 100: .4f}%')
    print(f'Validation Acc: {eval(bc_model, val_home_features, val_away_features, val_results) * 100: .4f}%')
    print('=' * 64)
except KeyboardInterrupt:
  print(f'Test Acc: {eval(bc_model, test_home_features, test_away_features, test_results) * 100: .4f}%')

Epoch: 1
Batch Avg. Loss:  4.4175
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 2
Batch Avg. Loss:  4.2273
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 3
Batch Avg. Loss:  4.0559
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 4
Batch Avg. Loss:  3.9364
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 5
Batch Avg. Loss:  3.8019
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 6
Batch Avg. Loss:  3.6854
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 7
Batch Avg. Loss:  3.6327
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 8
Batch Avg. Loss:  3.5759
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 9
Batch Avg. Loss:  3.5206
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 10
Batch Avg. Loss:  3.4773
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 11
Batch Avg. Loss:  3.4715
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 12
Batch Avg. Loss:  3.4454
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 13
Batch Avg. Loss

In [ ]:
bc_model.eval()
out = bc_model(train_home_features.reshape(-1, feature_size), train_away_features.reshape(-1, feature_size))
(out.argmax(dim=1) == 1).sum()

tensor(580)

# Linup

In [181]:
PlaysForEdgesDf.columns

Index(['gameId', 'playerId', 'gameDate', 'teamId', 'preGame_nGames',
       'preGame_nMinutes', 'playerName', 'teamName', 'src_nodeId',
       'src_nodeName', 'edge_type', 'dst_nodeId', 'dst_nodeName'],
      dtype='object')

In [182]:
FieldPlayerNodesDf.columns

Index(['player_optaId', 'postGame_subbedOut', 'postGame_RedCard',
       'postGame_minPlayed', 'preGame_side', 'preGame_line',
       'preGame_position', 'playerId', 'name', 'postGame_positions',
       'preGame_overall', 'preGame_potential', 'preGame_marketValueMilEuro',
       'preGame_preferredFoot', 'preGame_ageDays', 'postGame_error',
       'postGame_clearance', 'postGame_index', 'postGame_shots',
       'postGame_shots_on_target', 'postGame_shots_left_foot',
       'postGame_shots_right_foot', 'postGame_shots_head',
       'postGame_shots_other', 'postGame_goals', 'postGame_goals_left_foot',
       'postGame_goals_right_foot', 'postGame_goals_head',
       'postGame_goals_other', 'postGame_xG', 'postGame_cross',
       'postGame_cross_success', 'postGame_pass', 'postGame_pass_success',
       'postGame_pass_final_third', 'postGame_pass_final_third_success',
       'postGame_pass_forward', 'postGame_pass_forward_success',
       'postGame_dribble', 'postGame_dribble_success', 'po

In [183]:
TeamNodesDf.columns

Index(['postGame_side', 'postGame_goals_scored', 'postGame_goals_conceded',
       'postGame_index', 'postGame_total_shots', 'postGame_total_xg',
       'postGame_total_big_chances', 'postGame_total_key_passes',
       'competitionId', 'gameId', 'teamId', 'preGame_steph',
       'preGame_quality_overall', 'preGame_quality_attack',
       'preGame_quality_midfield', 'preGame_quality_defence',
       'preGame_proportion_pts', 'gameDate', 'teamName', 'competitionName',
       'nodeId', 'nodeName'],
      dtype='object')

In [184]:
GKNodesDf.columns

Index(['player_optaId', 'postGame_subbedOut', 'postGame_RedCard',
       'postGame_minPlayed', 'preGame_side', 'preGame_line',
       'preGame_position', 'playerId', 'name', 'preGame_overall',
       'preGame_potential', 'preGame_marketValueMilEuro',
       'preGame_preferredFoot', 'preGame_ageDays', 'preGame_xgpm',
       'preGame_xppm', 'postGame_error', 'postGame_clearance',
       'postGame_index', 'postGame_pickUp', 'postGame_punch', 'postGame_save',
       'gameId', 'teamId', 'teamName', 'gameDate', 'nodeId', 'nodeName'],
      dtype='object')

In [185]:
CompSeasonNodesDf.columns

Index(['seasonId', 'competitionId', 'competitionName', 'nodeId', 'nodeName'], dtype='object')

In [186]:
query = """
  select T.teamName,
         G.name || ' - ' || group_concat(FP.name, ' - ') as lineup,
         G.preGame_overall || ' - ' || group_concat(FP.preGame_overall, ' - ') as vg_rating,
         G.preGame_potential || ' - ' || group_concat(FP.preGame_potential, ' - ') as potential,
         G.preGame_marketValueMilEuro || ' - ' || group_concat(FP.preGame_marketValueMilEuro, ' - ') as market_value,
         G.preGame_ageDays || ' - ' || group_concat(FP.preGame_ageDays, ' - ') as age_days,
         G.preGame_xppm || ' - ' || group_concat(FP.preGame_xppm, ' - ') as xppm,
         G.preGame_xgpm || ' - ' || group_concat(FP.preGame_xgpm, ' - ') as xgpm,
         T.gameId,
         T.postGame_goals_scored as goals,
         T.competitionName as league,
         T.gameDate as date

  from FieldPlayerNodesDf as FP inner join (
    select *
    from TeamNodesDf
    order by gameDate, gameId, postGame_side desc
  ) as T
   on T.gameId = FP.gameId and FP.teamName = T.teamName
   inner join GKNodesDf G
   on G.teamName = T.teamName and T.gameId = G.gameId
  group by T.gameDate, T.gameId, T.postGame_side
"""
team_players_gameDf = ps.sqldf(query)
team_players_gameDf.to_csv('team_players_game.csv')
team_players_gameDf

,teamName,lineup,vg_rating,potential,market_value,age_days,xppm,xgpm,gameId,goals,league,date
0,Bristol City,Niki Maenpaa - Jack Hunt - Lloyd Kelly - Josh ...,68.0 - 70.0 - 60.0 - 70.0 - 72.0 - 72.0 - 69.0...,68.0 - 71.0 - 79.0 - 79.0 - 72.0 - 77.0 - 69.0...,0.0525 - 1.7 - 0.0375 - 2.7 - 2.2 - 3.5 - 1.1 ...,12263.0 - 10120.0 - 7264.0 - 8281.0 - 11467.0 ...,-0.127213322 - 0.129271668 - -0.109520788 - 0....,0.76310156 - -2.237213634 - -1.422035126 - 1.9...,3a604f5616b39eb17fc8d1eed07d5248e387bf400294b2...,3,England Championship,2018-08-21T18:45:00.000Z
1,Queens Park Rangers,Matt Ingram - Angel Rangel - Jake Bidwell - Jo...,64.0 - 73.0 - 68.0 - 71.0 - 75.0 - 70.0 - 73.0...,72.0 - 73.0 - 73.0 - 77.0 - 76.0 - 70.0 - 76.0...,0.05 - 1.1 - 1.0 - 2.8 - 6.5 - 1.2 - 5.0 - 7.0...,9012.0 - 13081.0 - 9284.0 - 9277.0 - 10230.0 -...,-0.361290054 - -0.202769976 - -0.37009074 - -0...,-2.8383644 - 0.47609611 - -2.437751452 - -1.25...,3a604f5616b39eb17fc8d1eed07d5248e387bf400294b2...,0,England Championship,2018-08-21T18:45:00.000Z
2,Hull City,David Marshall - Todd Kane - Eric Lichaj - Mar...,71.0 - 68.0 - 69.0 - 71.0 - 65.0 - 72.0 - 69.0...,71.0 - 73.0 - 69.0 - 74.0 - 77.0 - 77.0 - 82.0...,1.4 - 1.0 - 0.0825 - 3.0 - 0.085 - 3.5 - 2.0 -...,12222.0 - 9105.0 - 10869.0 - 9524.0 - 8024.0 -...,-0.052860584 - -0.163632598 - -0.383620298 - -...,-0.637533448 - -1.637829298 - -0.002807212 - -...,58b1242154c8055252582229abfc4680460278834c4433...,3,England Championship,2018-08-21T18:45:00.000Z
3,Rotherham United,Marek Rodak - Zak Vyner - Joe Mattock - Matthe...,65.0 - 63.0 - 66.0 - 67.0 - 65.0 - 65.0 - 66.0...,73.0 - 70.0 - 66.0 - 72.0 - 69.0 - 65.0 - 68.0...,0.06 - 0.045 - 0.0525 - 1.0 - 0.06 - 0.0375 - ...,7921.0 - 7769.0 - 10326.0 - 8576.0 - 9051.0 - ...,-0.056845074 - 0.031256748 - -0.056847772 - -0...,-0.298563454 - -0.493557718 - -0.298649904 - -...,58b1242154c8055252582229abfc4680460278834c4433...,2,England Championship,2018-08-21T18:45:00.000Z
4,Ipswich Town,Bartosz Bialkowski - Janoi Donacien - Jonas Kn...,75.0 - 64.0 - 70.0 - 60.0 - 67.0 - 71.0 - 66.0...,75.0 - 68.0 - 75.0 - 78.0 - 71.0 - 71.0 - 69.0...,4.5 - 0.05 - 2.0 - 0.0375 - 0.0825 - 1.6 - 0.0...,11370.0 - 9057.0 - 9471.0 - 6987.0 - 9645.0 - ...,-0.26088178 - -0.09205937 - -0.27144008 - -0.0...,-0.482914336 - 0.67430639 - 1.009222354 - 0.67...,c0c48eee0b1a42e0d84cb0a947fe2c64f9e1aa7015922f...,0,England Championship,2018-08-21T18:45:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...
8003,Levante,Dani Cárdenas - Oscar Duarte - Son - Tono - Go...,69.0 - 73.0 - 73.0 - 74.0 - 76.0 - 74.0 - 79.0...,77.0 - 73.0 - 74.0 - 74.0 - 77.0 - 79.0 - 79.0...,2.0 - 2.0 - 3.0 - 3.0 - 9.0 - 6.0 - 17.0 - 9.0...,8820.0 - 11675.0 - 9914.0 - 11518.0 - 10001.0 ...,-0.077330038 - -0.160066184 - -0.1942731 - -0....,-0.968012304 - -1.824673702 - -1.282755892 - -...,0dd5f2b145867283932a47fb03be7b9bc56d89f8250ece...,2,Spain Primera Division,2021-05-21T19:00:00.000Z
8004,Villarreal,Geronimo Rulli - Yeremi Pino - Carlos Bacca - ...,80.0 - 67.0 - 77.0 - 85.0 - 79.0 - 80.0 - 82.0...,80.0 - 84.0 - 77.0 - 85.0 - 79.0 - 86.0 - 82.0...,15.0 - 2.0 - 6.0 - 44.0 - 15.0 - 30.0 - 10.0 -...,10594.0 - 6789.0 - 12675.0 - 11724.0 - 11137.0...,-0.039779654 - 0.028538912 - 0.154979466 - -0....,-0.223861002 - 0.133822396 - 1.04837649 - 0.53...,0074aca0bc12614b3f47bc770421e7a25d1537424fd53a...,1,Spain Primera Division,2021-05-22T16:00:00.000Z
8005,Real Madrid,Thibaut Courtois - Karim Benzema - Éder Milita...,89.0 - 89.0 - 80.0 - 87.0 - 80.0 - 83.0 - 59.0...,91.0 - 89.0 - 87.0 - 87.0 - 90.0 - 89.0 - 79.0...,88.0 - 83.0 - 40.0 - 36.0 - 45.0 - 58.0 - 0.06...,10603.0 - 12208.0 - 8525.0 - 13039.0 - 7619.0 ...,0.57658046 - 0.630389486 - 0.26022381 - 0.4431...,1.652889838 - 1.693528026 - -0.034710492 - 0.7...,0074aca0bc12614b3f47bc770421e7a25d1537424fd53a...,2,Spain Primera Division,2021-05-22T16:00:00.000Z
8006,Valencia CF,Jasper Cillessen - Thierry Correia - Maximilia...,80.0 - 74.0 - 80.0 - 82.0 - 73.0 - 77.0 - 81.0...,80.0 - 80.0 - 85.0

In [187]:
ps.sqldf("""
  select age_days, lineup
  from team_players_gameDf
  where teamName = 'Real Madrid'
  order by date
""")

,age_days,lineup
0,9623.0 - 9744.0 - 11083.0 - 10481.0 - 9274.0 -...,Thibaut Courtois - Daniel Carvajal - Marcelo -...
1,9630.0 - 8318.0 - 10474.0 - 9708.0 - 9281.0 - ...,Thibaut Courtois - Álvaro Odriozola - Nacho Fe...
2,9634.0 - 10478.0 - 11094.0 - 9712.0 - 9285.0 -...,Thibaut Courtois - Nacho Fernandez - Marcelo -...
3,9637.0 - 9758.0 - 10481.0 - 9715.0 - 9288.0 - ...,Thibaut Courtois - Daniel Carvajal - Nacho Fer...
4,9644.0 - 8332.0 - 10488.0 - 9722.0 - 9295.0 - ...,Thibaut Courtois - Álvaro Odriozola - Nacho Fe...
5,9658.0 - 8346.0 - 11118.0 - 9736.0 - 9309.0 - ...,Thibaut Courtois - Álvaro Odriozola - Marcelo ...
6,9666.0 - 10510.0 - 11126.0 - 9744.0 - 9317.0 -...,Thibaut Courtois - Nacho Fernandez - Marcelo -...
7,9672.0 - 8360.0 - 7992.0 - 9750.0 - 10516.0 - ...,Thibaut Courtois - Álvaro Odriozola - Sergio R...
8,9680.0 - 8368.0 - 8000.0 - 9758.0 - 10524.0 - ...,Thibaut Courtois - Álvaro Odriozola - Sergio R...
9,9693.0 - 10723.0 - 12129.0 - 10551.0 - 9344.0 ...,Thibaut Courtois - Gareth Bale - Luka Modric -...


In [188]:
# week_plus = ps.sqldf("""
#                 select *,
#                        count(*) over (
#                            partition by teamName, league
#                            rows between unbounded preceding
#                            and current row
#                        ) as week
#                 from team_players_gameDf
#                 order by date
#             """)
# week_plus.to_csv('tpg.csv')

# %cd ../../../../../../../../
# tpg = pd.read_csv('tpg.csv')

In [189]:
query = """
  select T2.teamName as home_team, 
         T1.teamName as away_team,
         T2.lineup as home_lineup, 
         T1.lineup as away_lineup, 
         T2.goals as home_goal,         
         T1.goals as away_goal,
         T1.league,
         case 
            when (T2.goals - T1.goals) > 0 then 'win'
            when (T2.goals - T1.goals) = 0 then 'tie'
            when (T2.goals - T1.goals) < 0 then 'loss'
          end as result,
         T1.date,
         T1.gameId
  from team_players_gameDf T1
    inner join team_players_gameDf T2
     on T1.gameId = T2.gameId and T1.teamName <> T2.teamName
"""
_ = ps.sqldf(query)
matches = _[_.index % 2 == 0].reset_index().drop('index', axis=1)
matches.to_csv('matches.csv')
matches

,home_team,away_team,home_lineup,away_lineup,home_goal,away_goal,league,result,date,gameId
0,Queens Park Rangers,Bristol City,Matt Ingram - Angel Rangel - Jake Bidwell - Jo...,Niki Maenpaa - Jack Hunt - Lloyd Kelly - Josh ...,0,3,England Championship,loss,2018-08-21T18:45:00.000Z,3a604f5616b39eb17fc8d1eed07d5248e387bf400294b2...
1,Rotherham United,Hull City,Marek Rodak - Zak Vyner - Joe Mattock - Matthe...,David Marshall - Todd Kane - Eric Lichaj - Mar...,2,3,England Championship,loss,2018-08-21T18:45:00.000Z,58b1242154c8055252582229abfc4680460278834c4433...
2,Derby County,Ipswich Town,Scott Carson - Jayden Bogle - Craig Forsyth - ...,Bartosz Bialkowski - Janoi Donacien - Jonas Kn...,2,0,England Championship,win,2018-08-21T18:45:00.000Z,c0c48eee0b1a42e0d84cb0a947fe2c64f9e1aa7015922f...
3,Swansea City,Leeds United,Erwin Mulder - Kyle Naughton - Martin Olsson -...,Bailey Peacock-Farrell - Jamie Shackleton - Ba...,2,2,England Championship,tie,2018-08-21T18:45:00.000Z,d0cc49c3230e300b529b270951b3b70b3224481add8354...
4,Aston Villa,Brentford,Orjan Nyland - Axel Tuanzebe - Alan Hutton - G...,Daniel Bentley - Henrik Dalsgaard - Yoann Barb...,2,2,England Championship,tie,2018-08-22T18:45:00.000Z,174dba7291174b4dbbfa9ea12dd944bb45bdd8ed905524...
...,...,...,...,...,...,...,...,...,...,...
3999,Valencia CF,Eibar,Jasper Cillessen - Thierry Correia - José Gayà...,Marko Dmitrovic - Roberto Correa - José Ángel ...,4,1,Spain Primera Division,win,2021-05-16T16:30:00.000Z,c6cfcf8d8f019088926eb0b48fa4b5309a4176e6e3003d...
4000,Cádiz,Elche,David Gil - Álex Martín - Alfonso Espino - Jen...,Paulo Gazzaniga - Josan - Fidel Chaves - Diego...,1,3,Spain Primera Division,loss,2021-05-16T16:30:00.000Z,e465fbafec023f3e0c991b1860bd9f763bdd36b152d6c6...
4001,Levante,Cádiz,Dani Cárdenas - Oscar Duarte - Son - Tono - Go...,David Gil - Isaac Carcelén - Augusto Fernández...,2,2,Spain Primera Division,tie,2021-05-21T19:00:00.000Z,0dd5f2b145867283932a47fb03be7b9bc56d89f8250ece...
4002,Real Madrid,Villarreal,Thibaut Courtois - Karim Benzema - Éder Milita...,Geronimo Rulli - Yeremi Pino - Carlos Bacca - ...,2,1,Spain Primera Division,win,2021-05-22T16:00:00.000Z,0074aca0bc12614b3f47bc770421e7a25d1537424fd53a...


In [190]:
query = """
  select T2.teamName as home_team, 
         T1.teamName as away_team,
         T2.lineup as home_lineup, 
         T1.lineup as away_lineup, 
         T2.goals as home_goal,         
         T1.goals as away_goal,
         T1.league,
         T1.vg_rating as away_vg_rating,
         T1.potential as away_potential,
         T1.market_value as away_market_value,
         T1.age_days as away_age_days,
         T1.xppm as away_xppm,
         T1.xgpm as away_xgpm,
         T2.vg_rating as home_vg_rating,
         T2.potential as home_potential,
         T2.market_value as home_market_value,
         T2.age_days as home_age_days,
         T2.xppm as home_xppm,
         T2.xgpm as home_xgpm,
         case 
            when (T2.goals - T1.goals) > 0 then 'win'
            when (T2.goals - T1.goals) = 0 then 'tie'
            when (T2.goals - T1.goals) < 0 then 'loss'
          end as result,
         T1.date,
         T1.gameId
  from team_players_gameDf T1
    inner join team_players_gameDf T2
     on T1.gameId = T2.gameId and T1.teamName <> T2.teamName
"""
_ = ps.sqldf(query)
matches_player_features = _[_.index % 2 == 0].reset_index().drop('index', axis=1)
matches_player_features.to_csv('matches_with_player_features.csv')
matches_player_features

,home_team,away_team,home_lineup,away_lineup,home_goal,away_goal,league,away_vg_rating,away_potential,away_market_value,away_age_days,away_xppm,away_xgpm,home_vg_rating,home_potential,home_market_value,home_age_days,home_xppm,home_xgpm,result,date,gameId
0,Queens Park Rangers,Bristol City,Matt Ingram - Angel Rangel - Jake Bidwell - Jo...,Niki Maenpaa - Jack Hunt - Lloyd Kelly - Josh ...,0,3,England Championship,68.0 - 70.0 - 60.0 - 70.0 - 72.0 - 72.0 - 69.0...,68.0 - 71.0 - 79.0 - 79.0 - 72.0 - 77.0 - 69.0...,0.0525 - 1.7 - 0.0375 - 2.7 - 2.2 - 3.5 - 1.1 ...,12263.0 - 10120.0 - 7264.0 - 8281.0 - 11467.0 ...,-0.127213322 - 0.129271668 - -0.109520788 - 0....,0.76310156 - -2.237213634 - -1.422035126 - 1.9...,64.0 - 73.0 - 68.0 - 71.0 - 75.0 - 70.0 - 73.0...,72.0 - 73.0 - 73.0 - 77.0 - 76.0 - 70.0 - 76.0...,0.05 - 1.1 - 1.0 - 2.8 - 6.5 - 1.2 - 5.0 - 7.0...,9012.0 - 13081.0 - 9284.0 - 9277.0 - 10230.0 -...,-0.361290054 - -0.202769976 - -0.37009074 - -0...,-2.8383644 - 0.47609611 - -2.437751452 - -1.25...,loss,2018-08-21T18:45:00.000Z,3a604f5616b39eb17fc8d1eed07d5248e387bf400294b2...
1,Rotherham United,Hull City,Marek Rodak - Zak Vyner - Joe Mattock - Matthe...,David Marshall - Todd Kane - Eric Lichaj - Mar...,2,3,England Championship,71.0 - 68.0 - 69.0 - 71.0 - 65.0 - 72.0 - 69.0...,71.0 - 73.0 - 69.0 - 74.0 - 77.0 - 77.0 - 82.0...,1.4 - 1.0 - 0.0825 - 3.0 - 0.085 - 3.5 - 2.0 -...,12222.0 - 9105.0 - 10869.0 - 9524.0 - 8024.0 -...,-0.052860584 - -0.163632598 - -0.383620298 - -...,-0.637533448 - -1.637829298 - -0.002807212 - -...,65.0 - 63.0 - 66.0 - 67.0 - 65.0 - 65.0 - 66.0...,73.0 - 70.0 - 66.0 - 72.0 - 69.0 - 65.0 - 68.0...,0.06 - 0.045 - 0.0525 - 1.0 - 0.06 - 0.0375 - ...,7921.0 - 7769.0 - 10326.0 - 8576.0 - 9051.0 - ...,-0.056845074 - 0.031256748 - -0.056847772 - -0...,-0.298563454 - -0.493557718 - -0.298649904 - -...,loss,2018-08-21T18:45:00.000Z,58b1242154c8055252582229abfc4680460278834c4433...
2,Derby County,Ipswich Town,Scott Carson - Jayden Bogle - Craig Forsyth - ...,Bartosz Bialkowski - Janoi Donacien - Jonas Kn...,2,0,England Championship,75.0 - 64.0 - 70.0 - 60.0 - 67.0 - 71.0 - 66.0...,75.0 - 68.0 - 75.0 - 78.0 - 71.0 - 71.0 - 69.0...,4.5 - 0.05 - 2.0 - 0.0375 - 0.0825 - 1.6 - 0.0...,11370.0 - 9057.0 - 9471.0 - 6987.0 - 9645.0 - ...,-0.26088178 - -0.09205937 - -0.27144008 - -0.0...,-0.482914336 - 0.67430639 - 1.009222354 - 0.67...,73.0 - 62.0 - 70.0 - 68.0 - 73.0 - 66.0 - 70.0...,73.0 - 80.0 - 70.0 - 75.0 - 73.0 - 78.0 - 70.0...,2.4 - 0.0525 - 1.4 - 1.1 - 3.0 - 0.0975 - 1.6 ...,12041.0 - 6599.0 - 10770.0 - 8652.0 - 11699.0 ...,0.189542594 - -0.08425303 - 0.41049709 - -0.44...,0.26434415 - -0.062877308 - 3.503897654 - -1.6...,win,2018-08-21T18:45:00.000Z,c0c48eee0b1a42e0d84cb0a947fe2c64f9e1aa7015922f...
3,Swansea City,Leeds United,Erwin Mulder - Kyle Naughton - Martin Olsson -...,Bailey Peacock-Farrell - Jamie Shackleton - Ba...,2,2,England Championship,64.0 - 57.0 - 74.0 - 71.0 - 70.0 - 73.0 - 73.0...,75.0 - 78.0 - 74.0 - 79.0 - 70.0 - 75.0 - 73.0...,0.06 - 0.023 - 4.6 - 3.0 - 1.4 - 4.0 - 3.1 - 8...,7966.0 - 6892.0 - 10579.0 - 8298.0 - 10958.0 -...,0.142629276 - -0.01151058 - 0.840425594 - 0.00...,-1.977935036 - -0.108128658 - 6.923603458 - 4....,72.0 - 74.0 - 75.0 - 77.0 - 73.0 - 60.0 - 61.0...,73.0 - 74.0 - 75.0 - 77.0 - 77.0 - 77.0 - 72.0...,2.5 - 4.4 - 5.0 - 9.0 - 4.4 - 0.04 - 0.0375 - ...,10763.0 - 10875.0 - 11054.0 - 10455.0 - 9442.0...,0.092375074 - -0.138945214 - -0.020641144 - -0...,-0.313414424 - -1.39915354 - -0.841050428 - -1...,tie,2018-08-21T18:45:00.000Z,d0cc49c3230e300b529b270951b3b70b3224481add8354...
4,Aston Villa,Brentford,Orjan Nyland - Axel Tuanzebe - Alan Hutton - G...,Daniel Bentley - Henrik Dalsgaard - Yoann Barb...,2,2,England Championship,72.0 - 71.0 - 72.0 - 70.0 - 63.0 - 65.0 - 68.0...,78.0 - 71.0 - 76.0 - 75.0 - 80.0 - 76.0 - 76.0...,3.1 - 2.0 - 3.4 - 2.3 - 0.0725 - 0.08 - 1.3 - ...,9172.0 - 10619.0 - 9236.0 - 9305.0 - 7443.0 - ...,0.468892564 - 0.390362372 - 0.044812602 - 0.26...,3.962602

In [191]:
feature_types = [
  'vg_rating',
  'potential',
  'market_value',
  'age_days',
  'xppm',
  'xgpm']
sep = ' - '
corrupt = []
for f_type in feature_types:
  home_column = f'home_{f_type}'
  away_column = f'away_{f_type}'
  home_feature = matches_player_features[home_column].values.astype('str')
  away_feature = matches_player_features[away_column].values.astype('str')

  home_feature = np.char.split(home_feature, sep)
  away_feature = np.char.split(away_feature, sep)

  home_condition = np.array(list(map(len, home_feature)))
  away_condition = np.array(list(map(len, away_feature)))

  corrupt.append(np.argwhere(home_condition != 11))
  corrupt.append(np.argwhere(away_condition != 11))

from itertools import chain
corrupt_indcs = np.unique(np.array(list(chain.from_iterable(list(map(lambda z: z.reshape(-1).tolist(), corrupt))))))

dataset = matches_player_features.drop(corrupt_indcs).reset_index()

In [192]:
feature_types = [
  'vg_rating',
  'potential',
  'market_value',
  'age_days',
  'xppm',
  'xgpm']
sep = ' - '
corrupt = []
home_features = []
away_features = []
for f_type in feature_types:
  home_column = f'home_{f_type}'
  away_column = f'away_{f_type}'
  home_feature = dataset[home_column].values.astype('str')
  away_feature = dataset[away_column].values.astype('str')

  home_feature = np.stack(np.char.split(home_feature, sep)).astype(np.double)
  away_feature = np.stack(np.char.split(away_feature, sep)).astype(np.double)
  home_features.append(home_feature) 
  away_features.append(away_feature)




In [193]:
home_features = np.stack(home_features)
away_features = np.stack(away_features)
away_features = np.moveaxis(away_features, 0, 1)
home_features = np.moveaxis(home_features, 0, 1)
away_features = torch.from_numpy(away_features)
home_features = torch.from_numpy(home_features)
# feature shapes are N, Cin, L where N is the number of games, Cin
# is the input feature vector size and L is 11 for 11 players

gd = dataset.loc[:, ['home_goal']].values - dataset.loc[:, ['away_goal']].values
dummy = 9
gd[gd > 0] = dummy
gd[gd == 0] = dummy + 1
gd[gd < 0] = dummy + 2
y = torch.from_numpy(gd - dummy).reshape(-1)


# Set Agg Model

In [152]:
from torch.nn import Module, Linear, NLLLoss, Tanh, BatchNorm1d, Dropout, LogSoftmax, PairwiseDistance, Conv1d, ReLU, ModuleList
from torch.optim import Adam

In [173]:
class SetAgg(Module):
  def __init__(self, feature_size, player_hidden_size, team_hidden_size, dropout=0.4):
    super(SetAgg, self).__init__()
    self.dropout_rate = dropout
    self.regularizer = Dropout(self.dropout_rate)
    self.player_mlp_shape = [feature_size] + player_hidden_size
    self.team_mlp_shape = [self.player_mlp_shape[-1]] + team_hidden_size
    player_mlp_list = [
                      Conv1d(
                          in_channels=self.player_mlp_shape[i], 
                          out_channels=self.player_mlp_shape[i+1], 
                          kernel_size=1)
                      for i in range(len(self.player_mlp_shape[:-1]))
                      ]
    team_mlp_list = [
                      Linear(self.team_mlp_shape[i], self.team_mlp_shape[i+1])
                      for i in range(len(self.team_mlp_shape[:-1]))
                    ]
    self.player_transform = ModuleList(player_mlp_list)
    self.team_transform = (ModuleList(team_mlp_list))
    self.class_transform = Linear(self.team_mlp_shape[-1] * 2, 3)
    self.class_bn = BatchNorm1d(3)
    self.player_bn = ModuleList([
            BatchNorm1d(i)
            for i in self.player_mlp_shape
    ])
    self.team_bn = ModuleList([
            BatchNorm1d(i)
            for i in self.team_mlp_shape[1:]
    ])
    self.activation = ReLU()
    self.classifier = LogSoftmax(-1)

  def encode_team(self, x_players):
    # x_player is of shape N, C, L
    h_players = self.player_bn[0](x_players)
    for fc, bn in zip(self.player_transform, self.player_bn[1:]):
      h_players = fc(h_players)
      h_players = self.activation(h_players)
      h_players = bn(h_players)
      h_players = self.regularizer(h_players)
    
    h_team = h_players.sum(2)

    for fc, bn in zip(self.team_transform, self.team_bn):
      h_team = fc(h_team)
      h_team = self.activation(h_team)
      h_team = bn(h_team)
      h_team = self.regularizer(h_team)

    return h_team

  def forward(self, x_home_players, x_away_players):
    h_home = self.encode_team(x_home_players)
    h_away = self.encode_team(x_away_players)

    h = torch.cat((h_home, h_away), dim=1)
    h = self.class_transform(h)
    h = self.regularizer(h)
    return self.classifier(h)



In [203]:
sa_model = SetAgg(6, [16, 32], [64, 64], 0.5).double()
print(sa_model)
criterion = NLLLoss()
# sa_model(home_features, away_features)

SetAgg(
  (regularizer): Dropout(p=0.5, inplace=False)
  (player_transform): ModuleList(
    (0): Conv1d(6, 16, kernel_size=(1,), stride=(1,))
    (1): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
  )
  (team_transform): ModuleList(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
  )
  (class_transform): Linear(in_features=128, out_features=3, bias=True)
  (class_bn): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (player_bn): ModuleList(
    (0): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (team_bn): ModuleList(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [196]:
def train(model, home, away, results, optimizer, criterion):
  model.train()
  optimizer.zero_grad()
  output = model(home, away)
  loss = criterion(output, results)
  loss.backward()
  optimizer.step() 
  return loss.item()


@torch.no_grad()
def eval(model, home, away, results):
  model.eval()
  output = model(home, away).argmax(-1)
  correct = (output == results).sum().item()
  return (correct / results.shape[0])

In [179]:
home_features.shape

torch.Size([3462, 6, 11])

In [194]:
train_size = 2800
validation_size = 500
batch_size = 28
feature_size = 6
rand_indcs = torch.randperm(y.shape[0])
train_indcs = rand_indcs[:train_size]
val_indcs = rand_indcs[train_size: train_size + validation_size]
test_indcs = rand_indcs[train_size + validation_size:]
train_home_features = home_features[train_indcs, :, :].reshape(-1, batch_size, feature_size, 11)
train_away_features = away_features[train_indcs, :, :].reshape(-1, batch_size, feature_size, 11)
train_results = y[train_indcs].reshape(-1, batch_size)

val_home_features = home_features[val_indcs, :, :]
val_away_features = away_features[val_indcs, :, :]
val_results = y[val_indcs]
test_home_features = home_features[test_indcs, :, :]
test_away_features = away_features[test_indcs, :, :]
test_results = y[test_indcs]

In [206]:
optimizer = Adam(sa_model.parameters(), lr=1e-4, weight_decay=0)
try:
  for epoch in range(1500000):
    b_loss = 0
    for home, away, y_batch in zip(train_home_features, train_away_features, train_results):
      b_loss += train(sa_model, home, away, y_batch, optimizer, criterion)
    print(f'Epoch: {epoch + 1}')
    print(f'Batch Avg. Loss: {b_loss / batch_size: .4f}')
    print(f'Train Acc: {eval(sa_model, train_home_features.reshape(-1, feature_size, 11), train_away_features.reshape(-1, feature_size, 11), train_results.reshape(-1)) * 100: .4f}%')
    print(f'Validation Acc: {eval(sa_model, val_home_features, val_away_features, val_results) * 100: .4f}%')
    print('=' * 64)
except KeyboardInterrupt:
  print(f'Test Acc: {eval(sa_model, test_home_features, test_away_features, test_results) * 100: .4f}%')

Epoch: 1
Batch Avg. Loss:  3.6975
Train Acc:  52.9286%
Validation Acc:  48.4000%
Epoch: 2
Batch Avg. Loss:  3.6850
Train Acc:  53.1071%
Validation Acc:  48.6000%
Epoch: 3
Batch Avg. Loss:  3.6806
Train Acc:  53.2857%
Validation Acc:  48.4000%
Epoch: 4
Batch Avg. Loss:  3.6805
Train Acc:  53.1429%
Validation Acc:  48.4000%
Epoch: 5
Batch Avg. Loss:  3.6817
Train Acc:  53.2857%
Validation Acc:  48.6000%
Epoch: 6
Batch Avg. Loss:  3.6504
Train Acc:  53.1071%
Validation Acc:  48.4000%
Epoch: 7
Batch Avg. Loss:  3.6921
Train Acc:  52.9643%
Validation Acc:  48.4000%
Epoch: 8
Batch Avg. Loss:  3.6853
Train Acc:  53.2143%
Validation Acc:  48.6000%
Epoch: 9
Batch Avg. Loss:  3.6757
Train Acc:  53.2500%
Validation Acc:  48.8000%
Epoch: 10
Batch Avg. Loss:  3.6732
Train Acc:  53.2857%
Validation Acc:  48.8000%
Epoch: 11
Batch Avg. Loss:  3.7020
Train Acc:  53.2143%
Validation Acc:  49.0000%
Epoch: 12
Batch Avg. Loss:  3.6393
Train Acc:  53.2500%
Validation Acc:  48.8000%
Epoch: 13
Batch Avg. Loss